In [88]:
DEBUG = False
model_name = 'mnasnet_100'

In [89]:
from PIL import Image
import torchvision.transforms as transforms
import torch
import torchvision

In [90]:
PATH = 'model/' + model_name + '.pt'
model = torch.jit.load(PATH)
# saved torchscript model can't transfer from train mode to eval mode
# model.eval()

In [91]:
def recognize(image_path):
    """Print top 5 possiblilities of the given image.

    Args:
      image_path: the path of the image

    Returns:
    """
    img = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224,224)),
    ])
    tensor = transform(img).unsqueeze(0)
    
    if DEBUG:
        print(img.size)
        print(tensor.shape)

    with torch.no_grad():
        out = model(tensor)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    
    with open("imagenet_classes.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    for i in range(top5_prob.size(0)):
        print(categories[top5_catid[i]], top5_prob[i].item())

In [92]:
def accuracy():
    transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)

    # image dataset is imagenet
    # can be found https://www.kaggle.com/ifigotin/imagenetmini-1000
    testset = torchvision.datasets.ImageFolder(
    "image/imagenet-mini/val", transform=transform
)

    testloader = torch.utils.data.DataLoader(testset, batch_size=5, shuffle=True)

    images, labels = iter(testloader).next()

    correct = 0
    total = 0
    idx = 0
    with torch.no_grad():
        for images, labels in testloader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            idx = idx + 1
            if idx == 100:
                break
    print(correct / total)

In [93]:
accuracy()

0.474


In [94]:
recognize('image/dog.jpg')


Samoyed 0.8151047229766846
Pomeranian 0.02057637646794319
white wolf 0.014584108255803585
Persian cat 0.011715124361217022
Angora 0.009728971868753433
